In [22]:
import re
import tweepy
from tweepy import OAuthHandler
from textblob import TextBlob
  
# Brian to do: 
# read https://textblob.readthedocs.io/en/dev/ 
# Update broken part of this (tweepy api updated?)
# Note source: https://www.geeksforgeeks.org/twitter-sentiment-analysis-using-python/
class TwitterClient(object):
    '''
    Generic Twitter Class for sentiment analysis.
    '''
    def __init__(self):
        '''
        Class constructor or initialization method.
        '''
        # keys and tokens from the Twitter Dev Console
        consumer_key = "X"
        consumer_secret = "X"
        access_token = "X"
        access_token_secret = "X"
  
        # attempt authentication
        try:
            # create OAuthHandler object
            self.auth = OAuthHandler(consumer_key, consumer_secret)
            # set access token and secret
            self.auth.set_access_token(access_token, access_token_secret)
            # create tweepy API object to fetch tweets
            self.api = tweepy.API(self.auth)
        except:
            print("Error: Authentication Failed")
  
    def clean_tweet(self, tweet):
        '''
        Utility function to clean tweet text by removing links, special characters
        using simple regex statements.
        '''
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])", " ", tweet).split())
  
    def get_tweet_sentiment(self, tweet):
        '''
        Utility function to classify sentiment of passed tweet
        using textblob's sentiment method
        '''
        # create TextBlob object of passed tweet text sentiment.polarity method of TextBlob
        analysis = TextBlob(self.clean_tweet(tweet))
        # set sentiment
        if analysis.sentiment.polarity > 0:
            return 'positive'
        elif analysis.sentiment.polarity == 0:
            return 'neutral'
        else:
            return 'negative'
  
    def get_tweets(self, query, count = 10):
        '''
        Main function to fetch tweets and parse them.
        '''
        # empty list to store parsed tweets
        tweets = []
  
        try:
            # call twitter api to fetch tweets
            fetched_tweets = self.api.search_tweets(q = query, count = count)
  
            # parsing tweets one by one
            for tweet in fetched_tweets:
                # empty dictionary to store required params of a tweet
                parsed_tweet = {}
  
                # saving text of tweet
                parsed_tweet['text'] = tweet.text
                # saving sentiment of tweet
                parsed_tweet['sentiment'] = self.get_tweet_sentiment(tweet.text)
  
                # appending parsed tweet to tweets list
                if tweet.retweet_count > 0:
                    # if tweet has retweets, ensure that it is appended only once
                    if parsed_tweet not in tweets:
                        tweets.append(parsed_tweet)
                else:
                    tweets.append(parsed_tweet)
  
            # return parsed tweets
            return tweets
  
        except tweepy.TweepError as e:
            # print error (if any)
            print("Error : " + str(e))

            
def main():
    # creating object of TwitterClient Class
    api = TwitterClient()
    # calling function to get tweets
    tweets = api.get_tweets(query = '"Back The Blue"', count = 200) #Brian: Note small dataset might be skewing precentages
  
    # picking positive tweets from tweets
    ptweets = [tweet for tweet in tweets if tweet['sentiment'] == 'positive']
    # percentage of positive tweets
    print("Positive tweets percentage: {} %",'{0:.5g}'.format(100*len(ptweets)/len(tweets)))
    # picking negative tweets from tweets
    ntweets = [tweet for tweet in tweets if tweet['sentiment'] == 'negative']
    # percentage of negative tweets
    print("Negative tweets percentage: {} %",'{0:.5g}'.format(100*len(ntweets)/len(tweets)))
    # percentage of neutral tweets
    print("Neutral tweets percentage: {} % \
",'{0:.5g}'.format(100*(len(tweets) -(len( ntweets )+len( ptweets)))/len(tweets)))
  
    # printing first 5 positive tweets
    print("\n\nPositive tweets:")
    for tweet in ptweets[:10]:
        print(tweet['text'])
  
    # printing first 5 negative tweets
    print("\n\nNegative tweets:")
    for tweet in ntweets[:10]:
        print(tweet['text'])

        
if __name__ == "__main__":
    main()

Positive tweets percentage: {} % 23.636
Negative tweets percentage: {} % 10.909
Neutral tweets percentage: {} %  65.455


Positive tweets:
@3rdangelmessen1 About as much sense as reporting police to the police they back the blue lies
RT @EspositoforNY: I joined @leezeldin, on @DanClarkReports @NYNOW_PBS to discuss my experience in the NYPD &amp; how to secure our streets &amp; s…
@TheConservLady Ashley Babbit had white male rage too. Then our brave, brave police officers took care of that!  Back the blue!
@kendallybrown Thankfully that didn’t impact me personally but how many employees of 3rd Party Vendors including ho… https://t.co/EtBKm9xSsf
@Oregonian Imagine what that gun would have done to a cop.
Every single Republican candidate in Oregon wants everyb… https://t.co/ASeP7paiht
RT @Kynes99: If Liberal tactics to win marginal seats off Labor fail, they could end up with even less seats with little hope of winning ba…
If you back the blue and want criminal activity to go down, Josh w